In [89]:
import re
import matplotlib as mat; 
import pandas as pd

# Llegeix el fitxer en mode binari
with open('./Addresses.WBX', 'rb') as file:
    binary_data = file.read()
    
# Decodifiquem i ignorem si hi ha errors.
data = binary_data.decode('utf-8', errors='ignore')  

# Eliminem caracters no imprimibles. 
data_cleaned = re.sub(r'[^\x20-\x7E]+', ' ', data)

In [90]:
# Expressió regular per cercar els valors dintre de la data.
pattern = re.compile(r'group(?P<group>[^\s]+)\s+host(?P<host>[^\s]+).*?login(?P<login>[^\s]+)\s+note(?P<note>.*?)\s+pwd(?P<pwd>[^\s]+)', re.DOTALL)

# Llista per emmegatzemar els resultats
results = []

# A mesura que anem trobant les dades agrupem les informacións.
for match in pattern.finditer(data_cleaned):
    group_data = {
        'group': match.group('group'),
        'IP': match.group('host'),
        'Username': match.group('login'),
        'Device': match.group('note').strip(),
        'Password': match.group('pwd')
    }
    results.append(group_data)


In [83]:
# Mostrar los resultados de manera legible
# for result in results:
#     print(f"Group: {result['group']}")
#     print(f"Host: {result['IP']}")
#     print(f"Login: {result['Username']}")
#     print(f"Note: {result['Device']}")
#     print(f"Password: {result['Password']}")
#     print("-" * 40)

In [91]:
# Convertim el resultat en un df
df = pd.DataFrame(results)

### Ara acabem de polir les dades.

In [110]:
# Treiem els usuaris els numeros finals que ens ha agafat, treiem qualsevol numero del nom per cada string.
df['Username'] = df['Username'].apply(lambda x: x.rstrip('0123456789*&%,.()=/$<>-#+').strip())
df['group'] = df['group'].apply(lambda x: x.rstrip('0123456789*&%,.()=/$<>-#+').strip())

In [111]:
# Establim el port ssh per defecta 2274
df['Port'] = '2274'

In [112]:
# Treiem el port de la IP
df['IP'] = df['IP'].apply(lambda x: x.split(':')[0])

In [113]:
df.to_csv('info_mikrotik.csv', index = False)